Problem 1) We will create a kfold CV pipeline for the heart disease dataset you worked with during the midterm exam.

Problem 1a) Read in the data and drop the rows with missing values. Remember that the dataset does not contain the feature names! Separate out the feature matrix (X) and the target variable (y). What is the balance of this dataset (the baseline accuracy)? (2 points)


In [6]:
import pandas as pd
import numpy  as np

# Let's load the data - 0 points for this because they had to do it during the exam
feature_names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach',\
                'exang','oldpeak','slope','ca','thal']
label = 'num'
df = pd.read_csv('data/cleveland.data',header=None)
df.columns = feature_names + [label] 

df.replace(to_replace='?',value = np.nan, inplace=True) # replace '?' with nans
df.dropna(inplace=True) 
df.reset_index(drop=True, inplace=True) 

y = df[label]
df.drop(columns=[label],inplace=True)
print(df.head()) # 1 point for separating X and y

classes, counts = np.unique(y,return_counts=True)

print('balance:',np.max(counts/len(y))) # 1 point for calculating the balance


    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca thal  
0    3.0  0.0  6.0  
1    2.0  3.0  3.0  
2    2.0  2.0  7.0  
3    3.0  0.0  3.0  
4    1.0  0.0  3.0  
balance: 0.5387205387205387


Problem 1b) ML pipeline with logistic regression

Split your data in a stratified manner into `other` and `test` (20% in `test`) and then split `other` into 5 stratified folds. 4 of those folds will be used for training, the last fold will be CV. You'll need to loop through the 5 options the CV fold can be selected. (4 points)

Preprocess the data. Apply the OneHotEncoder and the StandardScaler to the appropriate columns. Make sure to fit_transform only train (4 out of the 5 folds). The CV and test sets should be transformed based on the preprocessor fitted to train. (2 points)

Train a logistic regression model with l1 regularization and tune the appropriate parameter. (4 points)

Repeat the procedure 10 times with 10 different random states and print the mean and std of the test accuracy score. Make sure to print the best parameters and check that the best values are not at the edge of your parameter space if possible. Check that your code is reproducable! That is, if you rerun the cell, you get back the exact same result. (3 points)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

cat_features = ['cp','slope','thal']

ct = make_column_transformer((OneHotEncoder(sparse=False,handle_unknown='ignore'), cat_features),
                             remainder=StandardScaler()) # 2 points

print(np.shape(ct.fit_transform(df)))

pipe = make_pipeline(ct, LogisticRegression(penalty='l1',solver='saga',max_iter=10000,multi_class='auto')) # 1 point

param_grid = {'logisticregression__C': [0.01,0.03,0.1,0.3,1,3]} # 1 point

def fit(df, y, random_state):
    X_other, X_test, y_other, y_test = train_test_split(df, y, random_state=random_state, test_size=0.2, stratify=y) # 2 points
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state) # 2 points
    grid = GridSearchCV(pipe, param_grid=param_grid,
                        cv=kf, return_train_score=True,iid=True) # 2 points
    grid.fit(X_other, y_other)
    print(grid.best_params_)# 1 point for printing best params
    print('CV score:',grid.best_score_) 
    print('test score:',grid.score(X_test, y_test))
    return grid.score(X_test, y_test)

test_scores = []
for i in range(10): # 1 point
    test_score = fit(df,y,i*42)
    test_scores.append(test_score)

print(np.mean(test_scores),np.std(test_scores)) # 1 point
    

(297, 20)
{'logisticregression__C': 0.3}
CV score: 0.5907172995780591
test score: 0.55
{'logisticregression__C': 3}
CV score: 0.5907172995780591
test score: 0.6
{'logisticregression__C': 0.3}
CV score: 0.5864978902953587
test score: 0.5666666666666667
{'logisticregression__C': 0.3}
CV score: 0.5907172995780591
test score: 0.5666666666666667
{'logisticregression__C': 0.3}
CV score: 0.6033755274261603
test score: 0.5833333333333334
{'logisticregression__C': 0.3}
CV score: 0.6075949367088608
test score: 0.5833333333333334
{'logisticregression__C': 0.3}
CV score: 0.5780590717299579
test score: 0.5833333333333334
{'logisticregression__C': 0.3}
CV score: 0.5907172995780591
test score: 0.5666666666666667
{'logisticregression__C': 0.3}
CV score: 0.5864978902953587
test score: 0.6333333333333333
{'logisticregression__C': 0.3}
CV score: 0.6033755274261603
test score: 0.6
0.5833333333333333 0.022360679774997883


Problem 1d) Train a random forest classifier and tune the appropriate parameters. Make sure to print the best parameters and check that the best values are not at the edge of your parameter space if possible. Repeat the procedure 10 times with 10 different random states and print the mean and std of the test accuracy score. Check that your code is reproducable! That is, if you rerun the cell, you get back the exact same result. (5 points)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

param_grid = {'randomforestclassifier__max_depth': [1,3,10,30],
             'randomforestclassifier__min_samples_split': [2,4,8,16]} # 2 points

def fit(df, y, random_state):
    X_other, X_test, y_other, y_test = train_test_split(df, y, random_state=random_state, test_size=0.2, stratify=y) # 2 points
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
    # RandomForestClassifier needs to be defined inside the function, random state needs to be set.
    pipe = make_pipeline(ct, RandomForestClassifier(n_estimators=100,random_state=random_state)) # 2 points
    grid = GridSearchCV(pipe, param_grid=param_grid,
                        cv=kf, return_train_score=True,iid=True) 
    grid.fit(X_other, y_other)
    print(grid.best_params_)# 1 point for printing best params
    print('CV score:',grid.best_score_) 
    print('test score:',grid.score(X_test, y_test))
    return grid.score(X_test, y_test)

test_scores = []
for i in range(10): 
    test_score = fit(df,y,i*42)
    test_scores.append(test_score)

print(np.mean(test_scores),np.std(test_scores)) 
    

{'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_split': 16}
CV score: 0.5780590717299579
test score: 0.55
{'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_split': 8}
CV score: 0.5949367088607594
test score: 0.5833333333333334
{'randomforestclassifier__max_depth': 3, 'randomforestclassifier__min_samples_split': 4}
CV score: 0.5780590717299579
test score: 0.5833333333333334
{'randomforestclassifier__max_depth': 3, 'randomforestclassifier__min_samples_split': 16}
CV score: 0.5907172995780591
test score: 0.5666666666666667
{'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_split': 8}
CV score: 0.5864978902953587
test score: 0.5833333333333334
{'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_split': 16}
CV score: 0.5864978902953587
test score: 0.5833333333333334
{'randomforestclassifier__max_depth': 30, 'randomforestclassifier__min_samples_split': 4}
CV score: 0.59071729957

Problem 1e) Train an SVC and tune the appropriate parameters. Make sure to print the best parameters and check that the best values are not at the edge of your parameter space if possible. Repeat the procedure 10 times with 10 different random states and print the mean and std of the test accuracy score. Check that your code is reproducable! That is, if you rerun the cell, you get back the exact same result. (4 points)

In [5]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(ct, SVC(kernel='rbf')) # 1 point

param_grid = {'svc__C': [0.01, 0.1, 1, 10, 100, 1000],
              'svc__gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} # 2 points

def fit(df, y, random_state):
    X_other, X_test, y_other, y_test = train_test_split(df, y, random_state=random_state, test_size=0.2, stratify=y) # 2 points
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state) 
    grid = GridSearchCV(pipe, param_grid=param_grid,
                        cv=kf, return_train_score=True,iid=True) # 1 points
    grid.fit(X_other, y_other)
    print(grid.best_params_)# 1 point for printing best params
    print('CV score:',grid.best_score_) 
    print('test score:',grid.score(X_test, y_test))
    return grid.score(X_test, y_test)

test_scores = []
for i in range(10): # 1 point
    test_score = fit(df,y,i*42)
    test_scores.append(test_score)

print(np.mean(test_scores),np.std(test_scores)) 


{'svc__C': 100, 'svc__gamma': 0.001}
CV score: 0.5822784810126582
test score: 0.6166666666666667
{'svc__C': 1, 'svc__gamma': 0.1}
CV score: 0.5907172995780591
test score: 0.5833333333333334
{'svc__C': 1000, 'svc__gamma': 0.001}
CV score: 0.5654008438818565
test score: 0.5666666666666667
{'svc__C': 100, 'svc__gamma': 0.001}
CV score: 0.6075949367088608
test score: 0.6
{'svc__C': 100, 'svc__gamma': 0.01}
CV score: 0.620253164556962
test score: 0.5
{'svc__C': 100, 'svc__gamma': 0.001}
CV score: 0.5949367088607594
test score: 0.55
{'svc__C': 100, 'svc__gamma': 0.1}
CV score: 0.5907172995780591
test score: 0.5166666666666667
{'svc__C': 10, 'svc__gamma': 0.01}
CV score: 0.569620253164557
test score: 0.55
{'svc__C': 10, 'svc__gamma': 0.01}
CV score: 0.5864978902953587
test score: 0.6166666666666667
{'svc__C': 1, 'svc__gamma': 0.1}
CV score: 0.6244725738396625
test score: 0.5333333333333333
0.5633333333333335 0.038586123009300755


Problem 1f) Compare the means and standard deviations of the three techniques. How many standard deviations above the baseline accuracy are the three models? How would you rank them with respect of accuracy? (3 points)

- Logistic regression: 0.58 +/- 0.02
- Random forest: 0.57 +/- 0.03
- SVC: 0.56 +/- 0.04

The balance is 0.54 and we did stratified splits so the same balance holds for each set. So the methods are 2, 1 and 0.5 sigmas above the baseline (actual numbers might vary because of random seeds). 2 points

Based on this, maybe logistic regression is better than the other two techniques but not by much. 1 point 

The results might vary because the different random states students might choose. Please check that their conclusions are correct based on their findings.